In [ ]:
from pystac import Item
from odc.stac import load
import folium
from ipyleaflet import basemaps

In [ ]:
row = 63
col = 20
year = 2024

url = (
    "https://dep-public-staging.s3.us-west-2.amazonaws.com/dep_s2_seagrass/0-0-1/"
    f"{row:03}/{col:03}/{year}/dep_s2_seagrass_{row:03}_{col:03}_{year}.stac-item.json"
)
item = Item.from_file(url)

data = load([item])

data

In [ ]:
# flipped = data.classification.isel(y=slice(None, None, -1))  # Flips the y axis
# center = flipped.odc.geobox.geographic_extent.centroid.coords[0][::-1]
# m = folium.Map(location=center, zoom_start=10, tiles=basemaps.Esri.WorldImagery)

# flipped.odc.add_to(m, name=variable, colormap="viridis")

In [ ]:
import boto3

bucket = "dep-public-staging"
prefix = "dep_s2_seagrass/0-0-1/"

s3 = boto3.client("s3")
result = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)

tiles = []
for obj in result.get("Contents", []):
    key = obj["Key"]
    if key.endswith(".stac-item.json") and f"/{year}/" in key:
        # Parse row and col from the filename or path
        # Example: dep_s2_seagrass/0-0-1/064/033/2024/dep_s2_seagrass_064_033_2024.stac-item.json
        parts = key.split("/")
        row, col = parts[-3], parts[-2]
        tiles.append((int(row), int(col)))

In [ ]:
data = data.compute()

from odc.geo.xr import write_cog

write_cog(data.classification, "marine_habitats.tif", overwrite=True)
write_cog(data.seagrass, "seagrass.tif", overwrite=True)
write_cog(data.seagrass_probability, "seagrass_probability.tif", overwrite=True)